In [50]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import time
import pandas as pd
import  pymongo
import numpy as np

In [51]:
executable_path = {"executable_path":"chromedriver.exe"}
browser = Browser('chrome', **executable_path, headless=True)
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.medical_info
collection = db.bariatras

In [52]:
url = "https://www.doctoralia.com.mx/buscar?filters%5Bservices%5D%5B0%5D=1189&q=Cirug%C3%ADa+bari%C3%A1trica&loc=Ciudad+de+M%C3%A9xico&page="

In [53]:
complete_data=[]
collection.drop()


In [54]:
page_counter = 1
for pages in range (0,10):
    try:
        browser.visit(url+str(page_counter))
        time.sleep(2)
        if "&page" in browser.url:
            response = browser.html
            soup = bs(response, 'html.parser')
            search_content = soup.find("ul", class_="new-search-list")
            results = search_content.find_all("li")
            for result in results:
                try:
                    ### Name
                    try:
                        doct_inf = {}
                        name = result.find("a", {"data-ga-label":"Doctor Name"})
                        clean_name = name.text.strip()
                    except:
                        clean_name= "NaN"
                    doct_inf["name"] = clean_name
                    ### Specialities
                    try:
                        specialties = result.find("h4", {"data-test-id":"doctor-specializations"})
                        clean_special = specialties.text.replace("\xa0", "").replace("…ver más", "")
                    except:
                        clean_special="NaN"
                    doct_inf["specialties"] = clean_special    
                    ### Address
                    try:
                        street = result.find("span", class_="street").text
                        city = result.find("span", class_="city").text
                        address = (f'{street}, {city}')
                    except:
                        address = "NaN"
                    doct_inf["address"] = address
                    ### link
                    try:
                        map_link = result.find("a", {"data-test-id":"address-map-link"})["href"]
                    except:
                        map_link = "NaN"
                    doct_inf["map_link"] = map_link
                    #### Price
                    try: 
                        price = result.find("p", class_="text-nowrap")
                        clean_price = int(price.text.replace("•", "").replace("$","").replace("desde","").replace(",","").strip())
                    except:
                        clean_price=0
                    doct_inf["price"] = clean_price
                    ### Opinion
                    try:
                        opinions = result.find_all("span", {"data-eecommerce-action":"product-click"})
                        clean_op = int(opinions[1].text.replace("opiniones", "").strip())
                    except:
                        clean_op = 0
                    doct_inf["opinions"] = clean_op
                    complete_data.append(doct_inf)
                except AttributeError:
                    print("Error-------------------------")
                    pass
            page_counter += 1
        

    except Exception as e:
        print(e)
        pass



In [119]:
doctor_df = pd.DataFrame(complete_data)
doctor_no_NAN = doctor_df.replace("NaN", np.nan)
doctor_df_clean = doctor_no_NAN.dropna(axis=0, how="any")
doctor_no_dup = doctor_df_clean.drop_duplicates("name")
final_df = doctor_no_dup.sort_values("price", ascending=False)
dicts = final_df.to_dict('records')
for i in dicts:
    collection.insert_one(i)